In [8]:
# import and load data
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import re
import pickle
import email
from tqdm import tqdm
import datetime
from dateutil import parser
import nltk


nltk.download('maxent_ne_chunker')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('words')




[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


True

In [13]:
with open('../data/final_body.pickle', 'rb') as file:
    dataset = pickle.load(file)

df = pd.DataFrame()
df['Body'] = dataset[0]
df = df[df['Body']!='']
df.shape

(1583, 1)

In [15]:
pd.set_option('display.max_colwidth', None)
df.head()

Body
0  This is pretty good God Bless the U S Air Force Someone pulled this off the Air Force Academy is very unofficial message board Written by a ' grad who apparently flies A What we all are REALLY thinking but a HOG driver says it Make no mistake about it this war is gonna be a Hawgdriver is dream no more of this sending GPS guided bombs from the ionosphere I want to camp out on the enemy is border I want to yell across the border in the immortal words of You tell 'em I am coming AND HELL'S COMING WITH ME I wanna punch in the face and say 'You gonna do something Or just stand there and bleed ' And then I wanna laugh maniacally as my shells decimate his camps I ai not talking about the Armor Piercing shells this time although the thought of poisoning their lungs if in the unlikely event they survived my attack with the dirty dust of spent uranium is quite refreshing that would make the cloud over New York seem like pure Oxygen I want High Explosive Incendiary rounds of them fired or hundred at a time like hundred grenades exploding all at once and that is just my jet the three coming with me brings that total to cans of CBU that is individual submunitions for them that is what I want I want Maverick missiles per jet that is of those things and if we run outta trucks and other small things to hit with those missiles I wanna find out what a maverick will do when it locks onto a terrorist and hits him at just over feet per second there might not be enough deceleration to detonate the thing but at that speed I do not think it would be necessary And I want pods of rockets hanging from my wings Seven white phosphorous and seven HEI I want the 'Willie ' to put a cloud of smoke to climb into the sky to let everyone following know that is where the gettin' is good And the HEI well I just want FRAG in the air tearing apart their greasy scumbag bodies the same way they tore into our nation and then we will start cleaning up with the almighty General Electric GAU A Avenger cannon what a perfect name AVENGER CANNON If that is all I had that is all I would want four hawgs with of our little friends lock and load hammer down But that is just my personal end here is what else I want I want and to take over CNN NBC ABC and every other news network to provide coverage of this war I want with his electronic chalkboard out there describing what is going on You see here across the top of the screen that ridge line is exactly where the attack is gonna come from you will see the Warthawgs come popping over them and unleash a fury that we have not seen since was on the prowl Speaking of that here they come and B These guys are great they remind me of linemen they do not get much press coverage but when they hit you man do you know it I want and belting out I'M PROUD TO BE AN ERICAN as the intro to Monday Night Air night would have a whole new meaning now I want to see air the HIT OF THE DAY Today is strike comes from a flight of two A Warthogs You will see here that some terrorist got the wild idea that he could shoot at these guys you can see the missile come up and totally miss the two jets and here you see as they roll in and unleash that awesome gun on the point of origin nothing left there now And that is our PLAY OF THE DAY I want in the field giving play by play descriptions I want of the Valkaries playing at full bore from every mountainside as we run in at feet I want WELCOME TO THE JUNGLE playing after the first bomb hits and when I am WINCHESTER ammunition I wanna land on  is personal airstrip grab him by his twisted dead neck and poke him in the eyes and say YOU JUST GOT KNOCKED OUT I want the NFL cheerleaders to send us off to war and the XFL cheerleaders to welcome us home And while we are at it I do not just want to beat the crap outta these scumbags I want to humiliate them too I want to see come outta retirement to start kicking some butt I want a cure for Alzheimer is right now to get back in working order and like says I want a 

In [19]:
msg =df.iloc[0]['Body']

def chunk_text(text, max_length=30):
        chunks = []
        text = text.split(" ")
        for i in range(0, len(text), max_length):
            chunks.append(' '.join(text[i:i + max_length]))
        return chunks


df['Body'] = df['Body'].apply(lambda text: chunk_text(text))
df = df.explode('Body')

In [20]:
df

,Body
0,This is pretty good God Bless the U S Air Force Someone pulled this off the Air Force Academy is very unofficial message board Written by a ' grad who
0,apparently flies A What we all are REALLY thinking but a HOG driver says it Make no mistake about it this war is gonna be a Hawgdriver is dream no
0,more of this sending GPS guided bombs from the ionosphere I want to camp out on the enemy is border I want to yell across the border in the immortal
0,words of You tell 'em I am coming AND HELL'S COMING WITH ME I wanna punch in the face and say 'You gonna do something Or just stand there and
0,bleed ' And then I wanna laugh maniacally as my shells decimate his camps I ai not talking about the Armor Piercing shells this time although the thought of poisoning
...,...
1701,address the hedges in the books will this need to be included in the CES true up Also wants to know if he should sign the letter Colleen I told
1701,you would anwser that question
1702,Yes CES Retail did take this gas for ch The Canonsburg office was responsible for the noms Yes the deal needs to be input into can input has copies of
1702,the monthly invoices from T F Operating for all three months Deal Volume MCF per month Price for MCF I would not worry about the Mcf issue here put the


In [21]:
# change the data into input and output sequences for our Encoder-decoder architectures
def split_sequence(df):
    '''
    To convert our data into encoder-decoder sequences we split sentences between contiguous spans of words.    
    sentences with < 5 words are dropped
    '''
    enc_seq = []
    dec_seq = []

    for text in tqdm(df.Body):
        sent = text.split()
        for i in range(5,len(sent)):
            enc_seq.append(''+' '.join(sent[:i]))
            dec_seq.append(' '.join(sent[i:i+5]))

    # put all in a dataframe
    data = pd.DataFrame()
    data['enc_seq'] = enc_seq
    data['dec_seq'] = dec_seq

    return data

In [22]:
data = split_sequence(df)
data.head()

100%|██████████| 6515/6515 [00:00<00:00, 13240.50it/s]


,enc_seq,dec_seq
0,This is pretty good God,Bless the U S Air
1,This is pretty good God Bless,the U S Air Force
2,This is pretty good God Bless the,U S Air Force Someone
3,This is pretty good God Bless the U,S Air Force Someone pulled
4,This is pretty good God Bless the U S,Air Force Someone pulled this


In [24]:
data.to_csv('../data/email_pairs.csv', index=False)

In [1]:
import os
from transformers import (
    T5ForConditionalGeneration,
    T5Tokenizer,
    TextDataset,
    DataCollatorForSeq2Seq,
    Trainer,
    TrainingArguments,
)
import datetime

c:\Users\Admin\anaconda3\envs\deeplearning\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [35]:
model_name = "t5-small"
timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
output_dir = f"../model/t5/fine-tuned-{timestamp}"

# Load model and tokenizer
tokenizer = T5Tokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token  # required
model = T5ForConditionalGeneration.from_pretrained(model_name)

c:\Users\Admin\anaconda3\envs\deeplearning\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Admin\.cache\huggingface\hub\models--t5-small. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download

In [36]:
# pip install SentencePiece 
import pandas as pd
from datasets import Dataset

data = pd.read_csv('../data/email_pairs.csv').dropna()
dataset = Dataset.from_pandas(data)

In [ ]:
def preprocess_function(examples):
    inputs = examples['enc_seq']
    targets = examples['dec_seq']
    model_inputs = tokenizer(inputs, max_length=30, truncation=True, padding="max_length")
    labels = tokenizer(targets, max_length=16, truncation=True, padding="max_length")
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [38]:
# Tokenize datasets
tokenized_data = dataset.map(preprocess_function, batched=True)

Map: 100%|██████████| 137809/137809 [00:50<00:00, 2711.44 examples/s]


In [41]:
split_data = tokenized_data.train_test_split(test_size=0.1, seed=42)

In [42]:
# Data collator for seq2seq
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

In [65]:
# Training arguments
training_args = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=2,
    save_strategy="epoch",
    logging_steps=500,
    fp16=True,  # IMPORTANT: for memory savings on 1050
    gradient_accumulation_steps=1,  # Effective batch size = 8
    save_total_limit=2,
    report_to="none"
)

In [66]:
# Trainer setup
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_data,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

C:\Users\Admin\AppData\Local\Temp\ipykernel_11820\4165279278.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train()


# trainer.train(resume_from_checkpoint=True)

Step,Training Loss
500,2.026200
1000,1.981900
1500,1.906000
2000,1.877300
2500,1.925500
3000,1.855500
3500,1.828100
4000,1.849400
4500,1.832200
5000,1.832500


In [ ]:
trainer.save_model(output_dir)
tokenizer.save_pretrained(output_dir)

print(f"✅ Model saved at {output_dir}")
